In [1]:
import json

# 假设你的 JSON 文件名为 'data.json'
with open('/home/chuannong1/KJY/MultiLLM/LLaMA-Factory/data/mllm_demo.json', 'r', encoding='utf-8') as file:
    data = json.load(file)  # 读取并解析 JSON 文件


In [2]:
len(data)

234022

In [5]:
from transformers import AutoTokenizer, AutoConfig

model_id = "/mnt/newdisk/KJY/MultiLLM/models/llava-1.5-7b-en/"

# 加载 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 加载 config
config = AutoConfig.from_pretrained(model_id)
print(config)


Exception: data did not match any variant of untagged enum ModelWrapper at line 277156 column 3

In [2]:
from transformers import pipeline, AutoProcessor
from PIL import Image    
import requests

model_id = "/mnt/newdisk/MultiLLM_models/llava-hf-7b/llava-1.5-7b-hf/"
pipe = pipeline("image-to-text", model=model_id)
image = Image.open("/home/chuannong1/KJY/MultiLLM/1.jpg").convert('RGB')

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.82it/s]
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:


# Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {
      "role": "user",
      "content": [
          {"type": "text", "text": "What does the label 15 represent? (1) lava (2) core (3) tunnel (4) ash cloud"},
          {"type": "image"},
        ],
    },
]
processor = AutoProcessor.from_pretrained(model_id)

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
print(outputs)

[{'generated_text': 'USER:  \nWhat does the label 15 represent? (1) lava (2) core (3) tunnel (4) ash cloud ASSISTANT: The label 15 represents the ash cloud.'}]


In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "/mnt/newdisk/KJY/swift/output/deepseek-vl-7b-chat/en/deepseek-vl-7b-chat/v0-20241114-155749/checkpoint-12723-merged/"
# model_id = "/mnt/newdisk/MultiLLM_models/llava-hf-7b/llava-1.5-7b-hf/"
model = LlavaForConditionalGeneration.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 8/8 [00:06<00:00,  1.19it/s]


In [13]:


# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Please describe the main pathological features of this patient."},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

image = Image.open("/home/chuannong1/KJY/MultiLLM/1.jpg").convert('RGB')
# image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(images=[image], text=[prompt], padding=True, return_tensors="pt")
inputs = inputs.to("cuda")
output_ids = model.generate(**inputs, max_new_tokens=200, do_sample=False)
# print(processor.decode(output[0], skip_special_tokens=True))


In [14]:
# Extract generated tokens (remove input part)
generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
# Decode generated tokens to readable text
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)[0]

In [15]:
output_text

'The patient in the image has a large, rapidly growing tumor in the right lobe of the liver, which is consistent with hepatocellular carcinoma (moderately differentiated). The liver capsule is intact. Immunohistochemistry results show that the tumor cells are PCK (+), CK8/18 (+), Heppar-1 (+), CK19 (-), CD34 blood vessels (+), CK7 (-), CK19 (-), P53 (-), and Ki67 about 10-20% (+).'

In [1]:
import torch
from transformers import AutoModelForCausalLM

from deepseek_vl.models import VLChatProcessor, MultiModalityCausalLM
from deepseek_vl.utils.io import load_pil_images


# specify the path to the model
model_path = "/mnt/newdisk/KJY/swift/output/deepseek-vl-7b-chat/en/deepseek-vl-7b-chat/v0-20241114-155749/checkpoint-12723-merged/"
vl_chat_processor: VLChatProcessor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

vl_gpt: MultiModalityCausalLM = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()

/home/chuannong1/anaconda3/envs/llama_factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python version is above 3.10, patching the collections module.


/home/chuannong1/anaconda3/envs/llama_factory/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:517: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Some kwargs in processor config are unused and will not have any effect: ignore_id, sft_format, mask_prompt, image_tag, num_image_tokens, add_special_token. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.62it/s]


In [2]:


conversation = [
    {
        "role": "User",
        "content": "<image_placeholder>Describe each stage of this image.",
        "images": ["/home/chuannong1/KJY/MultiLLM/1.jpg"]
    },
    {
        "role": "Assistant",
        "content": ""
    }
]

# load images and prepare for inputs
pil_images = load_pil_images(conversation)
prepare_inputs = vl_chat_processor(
    conversations=conversation,
    images=pil_images,
    force_batchify=True
).to(vl_gpt.device)

# run image encoder to get the image embeddings
inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

# run the model to get the response
outputs = vl_gpt.language_model.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=prepare_inputs.attention_mask,
    pad_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=512,
    do_sample=False,
    use_cache=True
)

answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
print(f"{prepare_inputs['sft_format'][0]}", answer)


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.

User: <image_placeholder>Describe each stage of this image.

Assistant: The image provided is a black and white radiological photograph, likely from a magnetic resonance imaging (MRI) machine, showing the liver tissue. The liver is the large, rounded structure in the center of the image. It has a capsule-like structure at the top and a denser, darker mass at the bottom. Surrounding the liver, there are smaller structures that appear to be lymph nodes. The liver tissue shows patchy areas of cirrhosis with nodular cirrhosis in the liver tissue and no cancer involvement in the surrounding liver tissue. No cancer metastasis was found in the lymph nodes. The image does not show any cancer cells or cancer metastasis in the liver tissue or lymph nodes around the liver.
